In [9]:
from datasets import load_from_disk, load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import numpy as np
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer, AutoModelForMaskedLM, BertConfig, BertModel, BertForPreTraining, BertForMaskedLM, AutoModel, PretrainedConfig, AutoConfig

import sys 
import os
sys.path.append(os.path.abspath(".."))

from collate import DataCollatorForLanguageModelingSpan


dataset = load_from_disk("../300")
dataset = dataset.remove_columns(["species_name", "__index_level_0__"])


tokenizer = AutoTokenizer.from_pretrained("gagneurlab/SpeciesLM", revision="downstream_species_lm")

# This way we don't load weights
# https://stackoverflow.com/questions/65072694/make-sure-bert-model-does-not-load-pretrained-weights
# TODO AutConfig or AutoModel? i guess it doesn't matter
config = PretrainedConfig.from_pretrained("togethercomputer/m2-bert-80M-2k")
config.vocab_size = tokenizer.vocab_size
model = BertForMaskedLM(config)

You are using a model of type m2_bert to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


In [ ]:
%env WANDB_PROJECT=singlesamplednam2

data_collator = DataCollatorForLanguageModelingSpan(tokenizer, mlm=True, mlm_probability = 0.02, span_length = 6)

training_args = TrainingArguments(
    output_dir="./results",
    
    max_steps=100000,
    
    seed=17,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=32,
    
    logging_strategy="steps",
    logging_steps=1,
    
    evaluation_strategy="no",
    
    #dataloader_num_workers=4,
    #dataloader_prefetch_factor=2,
    run_name="300set_bs1024_monarchhf",
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator
)

trainer.train()

env: WANDB_PROJECT=singlesamplednam2


/opt/modules/i12g/anaconda/envs/m2-mixer/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: luluhu (luluh). Use `wandb login --relogin` to force relogin


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,10.472300
2,10.360700
3,10.297500
4,10.240200
5,10.193900
6,10.141900
7,10.093200
8,10.042700
9,9.991700
10,9.949800


In [10]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4101, 768, padding_idx=0)
      (position_embeddings): Embedding(2048, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [8]:
tokenizer.vocab_size
config

PretrainedConfig {
  "_name_or_path": "togethercomputer/m2-bert-80M-2k",
  "alibi_starting_size": 2048,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "auto_map": {
    "AutoConfig": "configuration_bert.BertConfig",
    "AutoModelForSequenceClassification": "bert_layers.BertForMaskedLM",
    "AutoTokenizer": "bert-base-uncased"
  },
  "bidirectional": true,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "hyena_emb_dim": 5,
  "hyena_filter_dropout": 0.2,
  "hyena_filter_order": 128,
  "hyena_lr_pos_emb": 1e-05,
  "hyena_training_additions": false,
  "hyena_w": 10,
  "hyena_w_mod": 1,
  "hyena_wd": 0.1,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "long_conv_kernel_learning_rate": 0.001,
  "long_conv_l_max": 2048,
  "max_position_embeddings": 2048,
  "monarch_mlp_nblocks": 4,
  "num_attention_heads": 12,
  "num